In [11]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

In [17]:
# with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time

In [25]:
concert_name = []
concert_location = []
concert_date = []


with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as driver:
    driver.get("https://tickets.interpark.com/contents/genre/concert")

    # click "콘서트 전체보기"
    all_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="contents"]/article[1]/section/div/div/div/button[1]')))
    ActionChains(driver).click(all_button).perform()

    # scroll down inifinitely, with a limit of 60 seconds
    SCROLL_PAUSE_TIME = 4
    MAX_SCROLL_DURATION = 10  # 최대 60초 (1분)

    start_time = time.time()
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        current_time = time.time()
        if current_time - start_time > MAX_SCROLL_DURATION:
            print("최대 스크롤 시간 초과 - 중단")
            break

        # scroll down to the bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("더 이상 스크롤할 내용 없음 - 중단")
            break

        last_height = new_height
    # end of the scrolling


    # locate the concert container and append the concert information
    container = driver.find_element(By.XPATH, '//*[@id="contents"]/article[5]/section/div[2]/div[2]')
    a_tags = container.find_elements(By.TAG_NAME, "a")

    for i in range(1, len(a_tags)+1):
        concert_name.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[1]').text)
        concert_location.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[2]').text)
        concert_date.append(driver.find_element(By.XPATH, f'//*[@id="contents"]/article[5]/section/div[2]/div[2]/a[{i}]/ul/li[3]').text)

⏰ 최대 스크롤 시간 초과 - 중단
총 25개의 링크가 발견되었습니다.


In [27]:
len(concert_name), len(concert_location), len(concert_date)

(25, 25, 25)

In [ ]:

container = driver.find_element(By.XPATH, '//*[@id="contents"]/article[5]/section/div[2]/div[2]')
a_tags = container.find_elements(By.TAG_NAME, "a")
print(f"총 {len(a_tags)}개의 링크가 발견되었습니다.")

# container
# //*[@id="contents"]/article[5]/section/div[2]/div[2]

#  Format of first concert
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[1] 이름
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[2] 장소In
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[1]/ul/li[3] 날짜

#  format of second concert
# //*[@id="contents"]/article[5]/section/div[2]/div[2]/a[2]/ul/li[1]
